# Results

So the first step is to obtain the project, and map each run to exactly which model, on which dataset, which fold

In [2]:
import os, sys
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

In [154]:
from warnings import warn

In [13]:
import pandas as pd

In [3]:
from pykt.utils.wandb_utils import WandbUtils

In [4]:
wu = WandbUtils('letech', 'kt_toolkits')
wu.sweep_dict

Warning!! sweep falconcode_2_2_dkt_qid_4, was ran multiple times, taking the latest one :['bqtj3c4l', 'ouo2cgkb', '3wi059sz']
Warning!! sweep falconcode_2_2_dkt_qid_3, was ran multiple times, taking the latest one :['fm9vipzn', 'iah0pw1h', 'rpxmti17']
Warning!! sweep falconcode_2_2_dkt_qid_2, was ran multiple times, taking the latest one :['8lsdhg54', 'w7fya0bz', '1d5gtk7z']
Warning!! sweep falconcode_2_2_dkt_qid_1, was ran multiple times, taking the latest one :['0vuv0tui', 'edsbgzk1', 't1w6xio5']
Warning!! sweep falconcode_2_2_dkt_qid_0, was ran multiple times, taking the latest one :['68sluxig', 'vvkxc907', 'mcoykidl', 'dvw1l1d7']
self.sweep_dict is {'falconcode_2_2_dkt_qid_4': 'bqtj3c4l', 'falconcode_2_2_dkt_qid_3': 'fm9vipzn', 'falconcode_2_2_dkt_qid_2': '8lsdhg54', 'falconcode_2_2_dkt_qid_1': '0vuv0tui', 'falconcode_2_2_dkt_qid_0': '68sluxig'}


#### We obtain a list of the datasets we used and the models trained

In [128]:
datasets, models = set(), set()
sweep_names = wu.sweep_dict.keys()
for sn in sweep_names:
    parts = sn.split("_")
    datasets.add("_".join(parts[:3]))
    models.add("_".join(parts[3: parts.index("qid")]))
    
datasets, models

({'falconcode_2_2'}, {'dkt'})

#### Obtain for the given dataset, model, and folds, the best results

We also need to know for that given model, what are the hyperparameters, so w

In [146]:
def get_hyperparameter_names(selected_sweeps):
    _, model_configs = wu.get_df(selected_sweeps[0])
    rejected_paramters = ["model_name", "dataset_name", "save_dir", "fold", "seed"]
    hyperparameters = [k for k in model_configs if k not in rejected_paramters]
    return hyperparameters

In [147]:
def get_best_hyperparameters(grouped_results, hyperparams):
    best_hyperparams = grouped_results.groupby(hyperparams)["validauc"].mean().to_frame()
    best_hyperparams = best_hyperparams.sort_values(by="validauc", ascending=False)
    best_hyperparams = best_hyperparams.head(1).index
    best_hyperparams = best_hyperparams.to_frame().reset_index(drop=True).iloc[0].to_dict()
    query_best_hyperparams = " & ".join([f"({k}=={repr(v)})" for k, v in best_hyperparams.items()])
    return best_hyperparams#, query_best_hyperparams

In [159]:
def check_all_finnished(selected_sweeps):
    for ss in selected_sweeps:
        status = wu.get_sweep_info(ss)["state"]
        if status != "FINISHED":
            message = "Careful sweep " + ss + " hasn't finnished yet"
            warn(message)

In [160]:
from itertools import product

best_hyperparams = {}
for dataset_name, model_name in product(datasets, models):
    selected_sweeps = [sn for sn in sweep_names 
                       if dataset_name in sn and model_name in sn]
    check_all_finnished(selected_sweeps)
    grouped_results = pd.concat(wu.get_multi_df(selected_sweeps))
    hyperparameters = get_hyperparameter_names(selected_sweeps)
    bests = get_best_hyperparameters(grouped_results, 
                                     hyperparameters)
    best_hyperparams[(dataset_name, model_name)] = bests
    
best_hyperparams

/tmp/ipykernel_4821/1632450084.py:6: UserWarning: Careful sweep falconcode_2_2_dkt_qid_2 hasn't finnished yet
  warn(message)
/tmp/ipykernel_4821/1632450084.py:6: UserWarning: Careful sweep falconcode_2_2_dkt_qid_1 hasn't finnished yet
  warn(message)


{('falconcode_2_2', 'dkt'): {'dropout': 0.05,
  'emb_size': 64,
  'emb_type': 'qid',
  'learning_rate': 0.001}}

At the end here we have a mapping of the hyperparameters to rerun model training with, which we will save somewhere in a file or a dictionary. We can then pass this dictionary to another training script which will run the model trainings with the good hyperparameters